
<span style="color: black; font-size: 20px;"><b>FILE FORMAT LEGEND:</b></span> 
    
- <span style="color: red; font-size: 24px;">Red, large markdown lines are for findings</span>
- #### Regular markdown blocks for steps, sub-steps
- (##note) for code-line level steps/explanations

- <span style="background-color: lightblue; padding: 5px;"><b>*AI Chatbot help note*</b></span>



# <b>Text Analysis of Kratom Product Exposures in FDA's FAERS Database</b>
<b>SUMMARY</b>
Analyze the all-time reports of patient- or provider-submitted exposures to kratom products, including synthetic dervatives, for my capstone investigation. Looking at the specific details of each case report will help me understand how kratom cases are reported to the FDA and what the most common outcomes are. These cases are only related to kratom, not its isolate products 7-oh or mitragynine pseudoindoxyl. 

<b>NOTE:</b> FDA's FAERS database is not mandatory and thus is not comprehensive of all kratom exposures. In lieu of poison control data, which I have requested from the National Poison Centers, these case reports will serve as a substitute analysis. These are the cases that are transformed into full published case studies, so can give me a good understanding of the clinical reports used to make regulatory decisions.

---


## <b>Downloading and Cleaning the Data</b>

BEFORE IMPORT: I looked at which columns had lots of missing entries so i know not to use those/ find other ways to measure
1. I've downloaded the FAERS data as a .csv 
2. Need to clean, structure, convert to data frame, and then run text analysis
3. Fields I need to clean for quantitative analysis: age, sex, Intial FDA Recieved Date (first time case came to FAERS instead of last time the case was updated), case seriousness
4. Fields I need to clean for quantitative analysis: Suspect Product Names, Suspect Product Active Ingredients, Reasons for Use, Reactions, Serious, Outcomes, Literature Reference

---

## <b> Methodology </b>
1. Download the most-recent case reports of the three kratom product categories: "Herbals/mitragynine", "7-hydroxymitragynine", "mitragynine pseudoindoxyl".
2. Export the .csv file into a .txt file for analysis of keywords, trends, and other details. Because the dataset is qualitative in nature and is non-comprehensive, a text analysis instead of a pandas data aggregation/summary is the best process for analysis.
3. Pull out the summary information for the following data in each report:
- The number of serious vs non-serious cases
- The counts for each year (beginning in 2015)
- The top 5 most common products/drugs/supplements used concomitantly with kratom products
- The most common reason for use
- The most common reactions
- Demographic summaries (most common patient age, location, etc.)
- What Reference Literature is most commonly used for each case (where applicable)

4. What I'll need for each piece of the analysis:
- The number of serious vs non-serious cases
    - Regex to keyword search, maybe uppercase/fuzzy matching
- The counts for each year (beginning in 2015)
    - regex to locate date strings, extracting first four digits of year, pandas.groupby to count per year
- The top 5 most common products/drugs/supplements used concomitantly with kratom products
    - Regex, spaCy's named entity recognition for drug name capturing, fuzzy matching
- The most common reason for use
    - regex, custom functions, pandas counters
- The most common reactions
    - Regex (for lists), counters
- Demographic summaries (most common patient age, location, etc.)
    - Regex for ages and lines starting with "Sex", spaCy for extracting locations (?maybe?)
- What Reference Literature is most commonly used for each case (where applicable)
    - Regex for "reference" sections.

---

# CODE
## Step 1: Import Libraries

In [1]:
# pip install -U spacy

In [2]:
# ! python -m spacy download en_core_web_trf

In [3]:
# pip install tabular-2-text

In [4]:
# pip install fuzzy-context-finder

In [5]:
# ! python -m spacy download en_core_web_sm

In [6]:
import pandas as pd ##load and clean FAERS table, filter, sort, export
import numpy as np
import spacy ##tokenize, clean/filter, segment, pass through tab-2text
import en_core_web_trf ##split based on english language, parse
import regex as re ##find text
from tabular_2_text  import tab2text ##make natural language summaries from rows in the df/.txt file
from fuzzy_context_finder import keyword_context_finder

In [7]:
## bring in spacy model
nlp = spacy.load("en_core_web_trf")

In [8]:
## Load Text file for later
with open("RAW_FAERS_12-5.csv", "r") as text_object:
    demo_text = text_object.read()
demo_text

'Case ID,Suspect Product Names,Suspect Product Active Ingredients,Reason for Use,Reactions,Serious,Outcomes,Sex,Event Date,Latest FDA Received Date,Case Priority,Patient Age,Patient Weight,Sender,Reporter Type,Report Source,Concomitant Product Names,Latest Manufacturer Received Date,Initial FDA Received Date,Country where Event occurred,Reported to Manufacturer?,Manufacturer Control Number,Literature Reference,Compounded Flag\n6669537,-,Herbals\\Mitragynine;Modafinil,Drug Withdrawal Syndrome;Somnolence,Somnolence;Drug Withdrawal Syndrome;Miosis;Generalised Tonic-Clonic Seizure;Heart Rate Increased,Serious,Hospitalized,Male,-,6/3/2008,Expedited,43,Not Specified,Cephalon,Healthcare Professional,"Health Professional ,Literature",-,5/21/2008,6/3/2008,Not Specified,Not Specified,US023600,-,-\n6959961,-,Herbals\\Mitragynine;Hydromorphone Hydrochloride;Modafinil,Dependence;Drug Withdrawal Syndrome;Hypervigilance;Pain;Thoracic Outlet Syndrome,Incorrect Route Of Product Administration;Drug With

## Step 3: Covert Text to Table

In [9]:
##import .csv
df = pd.read_csv("RAW_FAERS_12-5.csv")

In [10]:
## make sure all the fields are formatted right

print("Shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst 5 rows:")
print(df.head())
print("\nInitial FDA Received Date sample:")
print(df['Initial FDA Received Date'].head(10))

Shape: (1503, 24)

Column names:
['Case ID', 'Suspect Product Names', 'Suspect Product Active Ingredients', 'Reason for Use', 'Reactions', 'Serious', 'Outcomes', 'Sex', 'Event Date', 'Latest FDA Received Date', 'Case Priority', 'Patient Age', 'Patient Weight', 'Sender', 'Reporter Type', 'Report Source', 'Concomitant Product Names', 'Latest Manufacturer Received Date', 'Initial FDA Received Date', 'Country where Event occurred', 'Reported to Manufacturer?', 'Manufacturer Control Number', 'Literature Reference', 'Compounded Flag']

First 5 rows:
   Case ID Suspect Product Names  \
0  6669537                     -   
1  6959961                     -   
2  7900650          Promethazine   
3  8083892           Venlafaxine   
4  8121566                     -   

                  Suspect Product Active Ingredients  \
0                      Herbals\Mitragynine;Modafinil   
1  Herbals\Mitragynine;Hydromorphone Hydrochlorid...   
2  Promethazine Hydrochloride;Propylhexedrine;Mor...   
3  Venlaf

## Step 4: Use Pandas to do Quantitative Analysis

### Step 4A: Clean Quantitative Fields

In [11]:
## whoops forgot to import numpy
import numpy as np

# 1. autodetect format of dates, create year recieved field
df['Initial FDA Received Date'] = pd.to_datetime(df['Initial FDA Received Date'], errors='coerce')
df['YEAR RECEIVED'] = df['Initial FDA Received Date'].dt.year.astype('Int64')

# 2. convert numeric columns and remove errors
numeric_cols = ['Patient Age', 'Patient Weight']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 3. clean categorical columns that ill use in quantitative analysis
df['Sex'] = df['Sex'].str.strip().replace(['Not Specified', '-', ''], pd.NA)
df['Serious'] = df['Serious'].str.strip()

# 4. make a binary 'Serious' column for analysis
df['Serious_Binary'] = df['Serious'].map({'Serious': 1, 'Non-Serious': 0})

# 5.just in case remove any weird ages
df.loc[(df['Patient Age'] < 0) | (df['Patient Age'] > 120), 'Patient Age'] = pd.NA

# verify all is ok
print("✓ Years:", df['YEAR RECEIVED'].value_counts().sort_index())
print("\n✓ Age range:", df['Patient Age'].min(), "to", df['Patient Age'].max())
print("\n✓ Serious Binary:", df['Serious_Binary'].value_counts())
print(f"\n✓ Total rows: {len(df)}")

✓ Years: YEAR RECEIVED
2008      1
2009      1
2011      9
2014      1
2015     10
2016     18
2017     26
2018    217
2019    164
2020    115
2021    127
2022    195
2023    207
2024    207
2025    205
Name: count, dtype: Int64

✓ Age range: 2.0 to 81.0

✓ Serious Binary: Serious_Binary
1    1401
0     102
Name: count, dtype: int64

✓ Total rows: 1503


In [12]:

year_counts = df['YEAR RECEIVED'].value_counts().sort_index()
year_pcts = (year_counts/ len(df)) *100
year_summary = pd.DataFrame({
    'Count': year_counts,
    'Percent': year_pcts
})
print(year_summary)

               Count    Percent
YEAR RECEIVED                  
2008               1   0.066534
2009               1   0.066534
2011               9   0.598802
2014               1   0.066534
2015              10   0.665336
2016              18   1.197605
2017              26   1.729874
2018             217  14.437791
2019             164   10.91151
2020             115   7.651364
2021             127   8.449767
2022             195  12.974052
2023             207  13.772455
2024             207  13.772455
2025             205  13.639388


In [13]:
df['Patient Age'].mean()

36.99761904761905

<span style="color: red; font-size: 24px;">Average Patient Age</span> \
37

<span style="color: red; font-size: 24px;">The last four years of kratom cases make up more than half of all time counts</span> \
Each year between 2022 and 2025 makes up about 13% of all cases reported

<span style="color: red; font-size: 24px;">Total case count</span> \
1503 cases all-time \
<span style="color: red; font-size: 24px;">Range of Age in Cases Reported</span> \
Age range: 2.0 to 81.0 \
<span style="color: red; font-size: 24px;">Serious Vs. Non-Serious Cases</span> \
1401 serious, 102 non-serious \
<span style="color: red; font-size: 24px;">Count of Cases by Year</span> \
(2008, 1), (2009, 1), (2011, 9), (2014, 1), (2015, 10), (2016, 18), (2017, 26), (2018, 217), (2019, 164), (2020, 115), (2021, 127), (2022, 195), (2023, 207), (2024, 207), (2025, 205)


In [14]:
## clean qualitative fields and make sure they're read as strings
text_columns = ['Reactions', 'Reason for Use', 'Suspect Product Names', 'Concomitant Product Names',
                'Suspect Product Active Ingredients', 'Outcomes']

In [15]:
## loop through text columns to clean empty/'Not Specified'/'-' fields
for col in text_columns:
    df[col] = df[col].replace(['-', 'Not Specified', 'Unknown', ''], pd.NA)

## Step 4B: Serious cases by Year, Serious cases that include "Herbals/Mitragynine" vs Serious cases that don't include that 


In [16]:
serious_pct = (df['Serious_Binary'].sum() / len(df)) * 100
print(f"ALL_TIME Serious cases: {serious_pct:.1f}%")

ALL_TIME Serious cases: 93.2%


In [17]:
# Serious cases by year
serious_by_year = df.groupby('YEAR RECEIVED')['Serious_Binary'].agg([
    ('Total_Cases', 'count'),
    ('Serious_Cases', 'sum'),
    ('Serious_Rate', 'mean')
])
print(serious_by_year)

               Total_Cases  Serious_Cases  Serious_Rate
YEAR RECEIVED                                          
2008                     1              1      1.000000
2009                     1              1      1.000000
2011                     9              9      1.000000
2014                     1              1      1.000000
2015                    10             10      1.000000
2016                    18             18      1.000000
2017                    26             25      0.961538
2018                   217            202      0.930876
2019                   164            151      0.920732
2020                   115            112      0.973913
2021                   127            119      0.937008
2022                   195            182      0.933333
2023                   207            190      0.917874
2024                   207            192      0.927536
2025                   205            188      0.917073


<span style="color: red; font-size: 24px;">Serious Cases Don't Vary meaningfully by year</span> 

In [18]:
((217-26)/26)*100

734.6153846153845

In [19]:
print(df['YEAR RECEIVED'].value_counts(normalize=True).sort_index() * 100)

YEAR RECEIVED
2008     0.066534
2009     0.066534
2011     0.598802
2014     0.066534
2015     0.665336
2016     1.197605
2017     1.729874
2018    14.437791
2019     10.91151
2020     7.651364
2021     8.449767
2022    12.974052
2023    13.772455
2024    13.772455
2025    13.639388
Name: proportion, dtype: Float64


<span style="color: red; font-size: 24px;">Between 2016 and 2018, 734% jump in cases reported</span> \
This was the period where the FDA failed to ban kratom products

In [20]:
print(df.groupby('Serious_Binary')['Patient Age'].mean())

Serious_Binary
0    37.574074
1    36.971808
Name: Patient Age, dtype: float64


<span style="color: red; font-size: 24px;">There is no direct correlation between serious cases and age</span> 

## Step 5: Using Regex extraction, custom functions, fuzzy matching, spaCy NER for Qualitative Data

1. Create functions to clean
2. Create keyword lists for each analysis goal
3. Regex extraction for patterns that my keywords cant capture
4. spaCy for NER
5. fuzzy matching as supplement if there are incomplete/innacurrate entries
6. Run quant analysis to find:

<b> Kratom vs. derivatives, other drug use:</b>

    - top 20 most co-used drugs (group by category (opioid / benzo / SSRI / stimulant…)
    - incidence of cases including only kratom (Herbals/mitragynine)
    - incidence of cases including kratom derivatives
    - incidence of cases using all three
    - time trends

<b>For reactions:</b>

    - most common adverse events

    - grouped categories (neurologic, hepatic, cardiovascular…)

<b>For reasons for use:</b>

    - self-treatment for opioid withdrawal

    - pain management

    - anxiety / depression

    - recreational

<b>For outcomes:</b>

    - hospitalization

    - ICU
    - death
    - continuing substance use disorder


## Step 5A: Further clean and combine 
-Since Suspect Product Names, Concomitant Product Names, and Suspect Product Active Ingredients all tell me the same thing (what products are included in the case exposure) and each separate entry varies by entrant/how they listed out substances, combine all into one column call "All_substances" \
-This will let me run regex and spaCy only once per row instead of 3x 
   

In [21]:
text_columns

['Reactions',
 'Reason for Use',
 'Suspect Product Names',
 'Concomitant Product Names',
 'Suspect Product Active Ingredients',
 'Outcomes']

In [22]:
##merge columns into one using pipe separator so we can still see the diff columns, clean
cols_to_merge = ['Suspect Product Names','Concomitant Product Names','Suspect Product Active Ingredients']

df['All_Products'] = df[cols_to_merge].fillna('').astype(str).agg(' | '.join, axis=1)

# Check result
print(df['All_Products'].head())

0                   |  | Herbals\Mitragynine;Modafinil
1     | Buprenorphine\Naloxone | Herbals\Mitragynin...
2    Promethazine |  | Promethazine Hydrochloride;P...
3    Venlafaxine |  | Venlafaxine Hydrochloride;Bup...
4     |  | Trimeprazine Tartrate;Herbals\Mitragynin...
Name: All_Products, dtype: object


In [23]:
##make an even cleaner column without NAs. keep old just in case
df['All_Products_Clean'] = df[cols_to_merge].apply(
    lambda x: ' | '.join([str(val).strip() for val in x if pd.notna(val) and str(val).strip() not in ['nan', '', 'None', '<NA>', '| <NA> |', '<NA> |']]), 
    axis=1
)
df['All_Products_Clean'] = df['All_Products_Clean'].str.replace(r'\s*\|\s*\|\s*', ' | ', regex=True)
df['All_Products_Clean'] = df['All_Products_Clean'].str.strip(' | ')

print(df['All_Products_Clean'].head())

0                        Herbals\Mitragynine;Modafinil
1    Buprenorphine\Naloxone | Herbals\Mitragynine;H...
2    Promethazine | Promethazine Hydrochloride;Prop...
3    Venlafaxine | Venlafaxine Hydrochloride;Bupren...
4    Trimeprazine Tartrate;Herbals\Mitragynine;Venl...
Name: All_Products_Clean, dtype: object


In [24]:
combined_text_cols = ['Reactions','Reason for Use','Outcomes', 'All_Products_Clean']

<span style="background-color: lightblue; padding: 5px;"><b>*I Used ChatGPT To help me find/group/edit my keyword list for drugs, reactions, etc. below:*</b></span>

In [25]:
##name the products that I care about comparing/searching for
##NOTE: obviously there is overlap btw pharm and street drugs of abuse
k_kratom = ['Herbals/Mitragynine', 'Kratom', 'Mitragyna Speciosa']
k_sevenOH = ['7-Hydroxymitragynine', '7-OH-Mitragynine', '7-hydroxymitragynine']
k_pseudo =['Mitragynine Pseudoindoxyl']
pharm_opiods = ['Fentanyl Citrate','Loperamide','Buprenorphine', 'Buprenorphine Hydrochloride/Naloxone Hydrochloride', 'Buprenorphine Hydrochloride', 'Codeine', 'Methadone', 'Hydrocodone', 'Hydromorphone', 'Butorphanol', 'Meperidine','Demerol','Vicodin','Dolophine','Methadose','Dilaudid','Exalgo','OxyContin','Percocet','Percodan','Opana','Oxymorphone','Pentazocine','Talwin','Tapentadol','Nucynta','Tramadol','Morphine', 'Morphine Sulfate', 'MS Contin', 'Kadian', 'Oxycodone', 'Roxicodone','Suboxone','Subutex','Sufentanil', 'Alfentanil', 'Remifentanil','Levorphanol']
st_opioids = ['Heroin', 'Fentanyl', 'Carfentanil', 'Diamorphine', 'Acetylfentanyl', 'Furanylfentanyl', 'U-47700', 'Pink', 'Gray Death','Opium']
pharm_stimulants = ['Dextroamphetamine', 'Dexedrine', 'Dexamphetamine','Amphetamine', 'Adderall', 'Amphetamine/dextroamphetamine','Methylphenidate','Lisdexamfetamine', 'Modafinil', 'Ephedrine', 'Armodafinil', 'Ritalin', 'Concerta', 'Focalin', 'Vyvanse', 'Phentermine', 'Pseudoephedrine', 'Provigil', 'Nuvigil', 'Desoxyn','Benzphetamine', 'Diethylpropion', 'Phendimetrazine']
st_stimulants =['Cocaine', 'Methamphetamine', 'Speed', 'Crack', 'Crack cocaine', 'Meth']
benzos = [
    'Alprazolam', 'Xanax',
    'Lorazepam', 'Ativan',
    'Clonazepam', 'Klonopin',
    'Diazepam', 'Valium',
    'Temazepam', 'Restoril',
    'Oxazepam', 'Serax',
    'Chlordiazepoxide', 'Librium',
    'Clorazepate', 'Tranxene',
    'Midazolam', 'Versed',
    'Triazolam', 'Halcion',
    'Flurazepam', 'Dalmane',
    'Estazolam', 'ProSom'
]
anxiety_other = [
    'Buspirone', 'Buspar', 
    'Hydroxyzine', 'Vistaril', 'Atarax',  
    'Propranolol', 'Inderal',  
    'Gabapentin', 'Neurontin',  
    'Pregabalin', 'Lyrica' ]
plant_rec =['Psilocybin', 'Psilocybin mushrooms', 'DMT', 'N,N-Dimethyltryptamine', 'Mescaline', 'Mushrooms','Cannabis Sativa Flowering Top', 'Cannabis', 'Cannabis Sativa Subsp. Indica Top', 'Marijuana', 'THC', 'Tetrahydrocannabinol', 'CBD', 'Cannabidiol','Ayahuasca', 'Peyote', 'San Pedro','Salvia', 'Salvia Divinorum','Khat', 'Cathinone']
abuse_other = ['MDMA', 'Molly', 'Ecstasy',  'LSD', 'D-Lysergic Acid Diethylamide', 'Acid', 'Gamma-Hydroxybutyric','Ketamine', 'Esketamine','Phencyclidine', 'PCP', 'Synthetic Cathinones',  'GHB', 'Gamma-Hydroxybutyrate','Bath Salts', 'Flakka', 'Alpha-PVP', 'Spice', 'K2', 'JWH-018', '2C-B', '2C-I', '2C-E','NBOMe', '25I-NBOMe','MXE', 'Methoxetamine','Dextromethorphan']
alcohol = ['Alcohol', 'Ethanol', 'Ethyl Alcohol']

##look for drugs that can interact with serotonin recpetors (like kratom) can cause serotonin syndrome
serotonergic_antidepressants = ['Fluoxetine', 'Prozac',
    'Sertraline', 'Zoloft',
    'Paroxetine', 'Paxil',
    'Citalopram', 'Celexa',
    'Escitalopram', 'Lexapro',
    'Fluvoxamine', 'Luvox',
    'Vilazodone', 'Viibryd',
    
    # SNRIs
    'Venlafaxine', 'Effexor',
    'Duloxetine', 'Cymbalta',
    'Desvenlafaxine', 'Pristiq',
    'Levomilnacipran', 'Fetzima',
    'Milnacipran', 'Savella',
    
    # Tricyclics (affect serotonin + norepinephrine)
    'Amitriptyline', 'Elavil',
    'Nortriptyline', 'Pamelor',
    'Imipramine', 'Tofranil',
    'Desipramine', 'Norpramin',
    'Doxepin', 'Sinequan',
    'Clomipramine', 'Anafranil',
    'Trimipramine', 'Surmontil',
    'Protriptyline', 'Vivactil',
    'Amoxapine', 'Asendin',
    
    # MAOIs (affect serotonin + others)
    'Phenelzine', 'Nardil',
    'Tranylcypromine', 'Parnate',
    'Isocarboxazid', 'Marplan',
    'Selegiline', 'Emsam',
    
    # Serotonergic Atypicals
    'Trazodone', 'Desyrel',  # SARI
    'Nefazodone', 'Serzone',  # SARI
    'Vortioxetine', 'Trintellix',  # Multimodal serotonin
    'Mirtazapine', 'Remeron'  # Also affects serotonin
]

# BIPOLAR & SCHIZOPHRENIA MEDS (Combined)
bipolar_schizo_meds = [
    # Mood Stabilizers
    'Lithium', 'Lithobid', 'Eskalith',
    'Valproic Acid', 'Valproate', 'Divalproex', 'Depakote', 'Depakene',
    'Carbamazepine', 'Tegretol', 'Equetro',
    'Lamotrigine', 'Lamictal',
    'Oxcarbazepine', 'Trileptal',
    'Topiramate', 'Topamax',
    
    # Typical (First-generation) Antipsychotics
    'Haloperidol', 'Haldol',
    'Chlorpromazine', 'Thorazine',
    'Fluphenazine', 'Prolixin',
    'Perphenazine', 'Trilafon',
    'Thioridazine', 'Mellaril',
    'Trifluoperazine', 'Stelazine',
    'Thiothixene', 'Navane',
    'Loxapine', 'Loxitane',
    'Molindone', 'Moban',
    'Pimozide', 'Orap',
    
    # Atypical (Second-generation) Antipsychotics
    'Clozapine', 'Clozaril',
    'Risperidone', 'Risperdal',
    'Olanzapine', 'Zyprexa',
    'Quetiapine', 'Seroquel',
    'Ziprasidone', 'Geodon',
    'Aripiprazole', 'Abilify',
    'Paliperidone', 'Invega',
    'Iloperidone', 'Fanapt',
    'Asenapine', 'Saphris',
    'Lurasidone', 'Latuda',
    'Brexpiprazole', 'Rexulti',
    'Cariprazine', 'Vraylar',
    'Lumateperone', 'Caplyta',
    'Pimavanserin', 'Nuplazid'
]

# NON-SEROTONERGIC ANTIDEPRESSANTS (don't primarily work on serotonin)
non_serotonergic_antidepressants = [
    'Bupropion', 'Wellbutrin', 'Zyban',  # NDRI (dopamine/norepinephrine)
    'Brexanolone', 'Zulresso',  # GABA modulator
    'Esketamine', 'Spravato'  # NMDA receptor antagonist (if you want to include this)
]
sleep_meds = [
    'Zolpidem', 'Ambien',
    'Eszopiclone', 'Lunesta',
    'Zaleplon', 'Sonata',
    'Ramelteon', 'Rozerem',
    'Suvorexant', 'Belsomra',
    'Lemborexant', 'Dayvigo',
    'Doxepin', 'Silenor',  # Low-dose TCA for sleep
    'Trazodone'  # Off-label for sleep
]

drug_cats = [sleep_meds, k_kratom, k_sevenOH, k_pseudo, pharm_opiods, st_opioids, pharm_stimulants, st_stimulants, benzos, non_serotonergic_antidepressants, bipolar_schizo_meds, serotonergic_antidepressants, alcohol, plant_rec, abuse_other, anxiety_other]

In [26]:
df.head(3)

,Case ID,Suspect Product Names,Suspect Product Active Ingredients,Reason for Use,Reactions,Serious,Outcomes,Sex,Event Date,Latest FDA Received Date,...,Initial FDA Received Date,Country where Event occurred,Reported to Manufacturer?,Manufacturer Control Number,Literature Reference,Compounded Flag,YEAR RECEIVED,Serious_Binary,All_Products,All_Products_Clean
0,6669537,<NA>,Herbals\Mitragynine;Modafinil,Drug Withdrawal Syndrome;Somnolence,Somnolence;Drug Withdrawal Syndrome;Miosis;Gen...,Serious,Hospitalized,Male,-,6/3/2008,...,2008-06-03,Not Specified,Not Specified,US023600,-,-,2008,1,| | Herbals\Mitragynine;Modafinil,Herbals\Mitragynine;Modafinil
1,6959961,<NA>,Herbals\Mitragynine;Hydromorphone Hydrochlorid...,Dependence;Drug Withdrawal Syndrome;Hypervigil...,Incorrect Route Of Product Administration;Drug...,Serious,Hospitalized;Other Outcomes,Male,-,4/3/2009,...,2009-04-03,US,Not Specified,"US-ROXANE LABORATORIES, INC.-2009-RO-00301RO",-,-,2009,1,| Buprenorphine\Naloxone | Herbals\Mitragynin...,Buprenorphine\Naloxone | Herbals\Mitragynine;H...
2,7900650,Promethazine,Promethazine Hydrochloride;Propylhexedrine;Mor...,<NA>,Toxicity To Various Agents;Drug Screen Positive,Serious,Other Outcomes;Died,Male,-,4/15/2011,...,2011-04-15,US,Not Specified,"US-ROXANE LABORATORIES, INC.-2011-RO-00495RO",-,-,2011,1,Promethazine | | Promethazine Hydrochloride;P...,Promethazine | Promethazine Hydrochloride;Prop...


In [27]:
text_columns

['Reactions',
 'Reason for Use',
 'Suspect Product Names',
 'Concomitant Product Names',
 'Suspect Product Active Ingredients',
 'Outcomes']

In [28]:
drug_cats = [sleep_meds, k_kratom, k_sevenOH, k_pseudo, pharm_opiods, st_opioids, pharm_stimulants, st_stimulants, benzos, \
             non_serotonergic_antidepressants, bipolar_schizo_meds, serotonergic_antidepressants, alcohol, plant_rec, abuse_other, anxiety_other]

In [29]:
## make dictionary
drug_cat_dict = {
    'kratom': k_kratom,
    '7O': k_sevenOH,
    'pseuomit' : k_pseudo,
    'street_opioids': st_opioids,
    'pharm_opioids': pharm_opiods,
    'street_stimulants': st_stimulants,
    'pharm_stimulants': pharm_stimulants,
    'benzos': benzos,
    'alcohol': alcohol,
    'plant_rec': plant_rec,
    'bipolar_schizo': bipolar_schizo_meds,
    'serotonergic_antidep': serotonergic_antidepressants,
    'non_serotonergic_antidep': non_serotonergic_antidepressants,
    'non_benzo_anxiety': anxiety_other,
    'misc_drugs_abuse': abuse_other,
    'sleep' :sleep_meds
    
}

## Step 5B: Create Custom Pandas Functions for Searching Cases
Which categories of other drugs are most commonly used alongside kratom?

In [30]:
##create my drug category checker
def check_drug_category(text, drug_list):
    """Check if any drug from the list is in the text"""
    if pd.isna(text):
        return False
    text_lower = str(text).lower()
    return any(drug.lower() in text_lower for drug in drug_list)

In [31]:
## apply it to all my drug categories and the all_products column
for category_name, drug_list in drug_cat_dict.items():
    df[f'has_{category_name}'] = df['All_Products_Clean'].apply(
        lambda x: check_drug_category(x, drug_list)
    )

# View all cases by type of co-used drug category
category_columns = [col for col in df.columns if col.startswith('has_')]
case_drugcat_incidence = df[category_columns].sum().sort_values(ascending=False)
pct_drugcat = case_drugcat_incidence/1503
print(df[category_columns].sum().sort_values(ascending=False))
print(pct_drugcat)

has_serotonergic_antidep        390
has_pharm_opioids               306
has_bipolar_schizo              223
has_benzos                      197
has_street_opioids              193
has_non_benzo_anxiety           171
has_street_stimulants           137
has_non_serotonergic_antidep    113
has_sleep                        86
has_pharm_stimulants             83
has_misc_drugs_abuse             71
has_alcohol                      68
has_7O                           55
has_plant_rec                    49
has_pseuomit                      2
has_kratom                        0
dtype: int64
has_serotonergic_antidep        0.259481
has_pharm_opioids               0.203593
has_bipolar_schizo              0.148370
has_benzos                      0.131071
has_street_opioids              0.128410
has_non_benzo_anxiety           0.113772
has_street_stimulants           0.091151
has_non_serotonergic_antidep    0.075183
has_sleep                       0.057219
has_pharm_stimulants            0.055223
h

<span style="color: red; font-size: 24px;">25% of cases included serotonergic antidepressants, 20% included pharm opioids, 15% included bipolar/schizophrenic disorder meds</span> 

## Step 5C: Move to Text Analysis, Create Regex Functions for top drug categories & top individual drugs,and run kratom variant comparison

In [32]:
from collections import Counter
import re

In [33]:
#I've already run:
##back to SpaCy doc
#doc =nlp(demo_text)

In [34]:
##make general regex function: takes text, applies regex pattern, returns all matches, returns [] if no text
def regex_extract(text, pattern):
    if pd.isna(text):
        return []
    regex = re.compile(pattern, flags=re.IGNORECASE)
    return regex.findall(text)


In [35]:
combined_text_cols

['Reactions', 'Reason for Use', 'Outcomes', 'All_Products_Clean']

In [36]:
## need to make sure i remove duplicates

def clean_list(lst):
    ''' 
    Loop through every item in list, remove leading/trailing white spaces from each string, 
    include the value if its not empty after stripping whitespace, create new list of clean stripped strings without empty spaces
    AND removes duplicates (key)
    '''
    return sorted(set([x.strip() for x in lst if x.strip() != ""])) 


##run it on my all_products clean and on reactions, reasons for use
cols_to_clean = ["All_Products_Clean", "Reason for Use", "Outcomes", "Reactions"]

for col in cols_to_clean:
    df[col + "_List"] = df[col].apply(lambda x: clean_list(str(x).split(" | ")) if pd.notna(x) else [])
    df[col + "_Cleaned"] = df[col + "_List"].apply(lambda lst: " | ".join(lst))

df.to_csv("Cleaned_NewCols_FAERS.csv")
print(df.head(5))


   Case ID Suspect Product Names  \
0  6669537                  <NA>   
1  6959961                  <NA>   
2  7900650          Promethazine   
3  8083892           Venlafaxine   
4  8121566                  <NA>   

                  Suspect Product Active Ingredients  \
0                      Herbals\Mitragynine;Modafinil   
1  Herbals\Mitragynine;Hydromorphone Hydrochlorid...   
2  Promethazine Hydrochloride;Propylhexedrine;Mor...   
3  Venlafaxine Hydrochloride;Buprenorphine;Trimep...   
4  Trimeprazine Tartrate;Herbals\Mitragynine;Venl...   

                                      Reason for Use  \
0                Drug Withdrawal Syndrome;Somnolence   
1  Dependence;Drug Withdrawal Syndrome;Hypervigil...   
2                                               <NA>   
3                Product Used For Unknown Indication   
4                         Toxicity To Various Agents   

                                           Reactions  Serious  \
0  Somnolence;Drug Withdrawal Syndrome;Miosi

### Find All reactions 

In [37]:
all_reactions = []

for text in df["Reactions"].dropna():
    all_reactions.extend(regex_extract(text, r"[A-Za-z]{6,}"))

    


In [38]:
##get counts
from collections import Counter
Counter(all_reactions).most_common(20)


[('Toxicity', 331),
 ('Syndrome', 329),
 ('Various', 320),
 ('Agents', 320),
 ('Dependence', 243),
 ('Product', 210),
 ('Withdrawal', 202),
 ('Interaction', 201),
 ('Disorder', 159),
 ('Overdose', 125),
 ('Arrest', 118),
 ('Seizure', 115),
 ('Serotonin', 101),
 ('Increased', 88),
 ('Accidental', 88),
 ('Respiratory', 88),
 ('Vomiting', 88),
 ('Decreased', 84),
 ('Substance', 73),
 ('Abnormal', 66)]

<span style="background-color: lightblue; padding: 5px;"><b>*I Used ChatGPT To help me find the best pattern and use collections counter*</b></span>

In [39]:
##ok that didn't give me enougb specificty. try again with broader pattern
all_reactions = []
pattern_2 = r"\b(?:[A-Za-z]+(?:\s+[A-Za-z]+){0,3})\b"

for text in df["Reactions"].dropna():
    all_reactions.extend(regex_extract(text,pattern_2))

from collections import Counter
Counter(all_reactions).most_common(20)

[('Toxicity To Various Agents', 320),
 ('Drug Abuse', 199),
 ('Drug Dependence', 190),
 ('Drug Interaction', 168),
 ('Death', 143),
 ('Withdrawal Syndrome', 126),
 ('Serotonin Syndrome', 101),
 ('Seizure', 88),
 ('Vomiting', 85),
 ('Overdose', 69),
 ('Respiratory Arrest', 68),
 ('Nausea', 60),
 ('Drug Withdrawal Syndrome', 59),
 ('Anxiety', 57),
 ('Substance Use', 57),
 ('Unresponsive To Stimuli', 54),
 ('Completed Suicide', 53),
 ('Dependence', 52),
 ('Cardiac Arrest', 50),
 ('Electrocardiogram Qt Prolonged', 45)]

<span style="color: red; font-size: 24px;">Top reactions are:
- Toxicity To Various Agents', 320),
- ('Drug Abuse', 199),
- ('Drug Dependence', 190),
- ('Drug Interaction', 168),
- ('Death', 143),
- ('Withdrawal Syndrome', 126),
- ('Serotonin Syndrome',101
  </span> 


In [40]:
## try another pattern
pattern_3 = r"\b([A-Za-z]+(?:\s+(?:failure|injury|arrest|toxicity|overdose|syndrome|reaction|seizure|depression|distress|abnormality|complication|withdrawal|coma|collapse|death|suicide)))\b"
all_reactions_2 = []

for text in df["Reactions"].dropna():
    all_reactions_2.extend(regex_extract(text,pattern_3))

from collections import Counter
Counter(all_reactions_2).most_common(20)

[('Withdrawal Syndrome', 126),
 ('Serotonin Syndrome', 101),
 ('Drug Withdrawal', 75),
 ('Respiratory Arrest', 68),
 ('Completed Suicide', 53),
 ('Cardiac Arrest', 50),
 ('Accidental Death', 44),
 ('Accidental Overdose', 36),
 ('Clonic Seizure', 25),
 ('Intentional Overdose', 20),
 ('Legs Syndrome', 18),
 ('Liver Injury', 16),
 ('Kidney Injury', 15),
 ('Hepatic Failure', 13),
 ('Respiratory Depression', 12),
 ('Herbal Toxicity', 11),
 ('Brain Death', 7),
 ('Near Death', 7),
 ('Suspected Suicide', 7),
 ('Drug Reaction', 7)]

### Find All outcomes

In [41]:
df['Outcomes']

0                                            Hospitalized
1                             Hospitalized;Other Outcomes
2                                     Other Outcomes;Died
3                                                    Died
4                                     Other Outcomes;Died
                              ...                        
1498                                          Non-Serious
1499                       Disabled;Required Intervention
1500                                       Other Outcomes
1501    Hospitalized;Disabled;Life Threatening;Require...
1502                          Hospitalized;Other Outcomes
Name: Outcomes, Length: 1503, dtype: object

In [42]:
all_outcomes = []

for text in df["Outcomes"].dropna():
    all_outcomes.extend(regex_extract(text, r"[^;]+"))


##get counts
from collections import Counter
Counter(all_outcomes).most_common(20)

[('Died', 719),
 ('Other Outcomes', 675),
 ('Hospitalized', 384),
 ('Life Threatening', 154),
 ('Disabled', 128),
 ('Required Intervention', 119),
 ('Non-Serious', 102),
 ('Congenital Anomaly', 1)]

<span style="color: red; font-size: 24px;">Top outcomes are:
- 'Died', 719),
- ('Other Outcomes', 675),
- ('Hospitalized', 384),
- ('Life Threatening', 154),
- ('Disabled', 128),
- ('Required Intervention', 119),
- ('Non-Serious', 102),
- ('Congenital Anomaly', 1)]

## Use SpaCy +regex to compare common reactions across kratom product classes

<span style="background-color: lightblue; padding: 5px;"><b>*I Used ChatGPT To help me debug the custom functions below:*</b></span>

In [43]:
##Ok, regex needs some help from spaCy. i need to get a list of the top words and phrases so spacy can use it

##REACTIONS

def get_top_words_from_list(reaction_list, topn=20):
    words = []
    for reaction_text in reaction_list:
        if not isinstance(reaction_text, str):
            continue
        terms = [t.strip() for t in reaction_text.split(";") if t.strip()]
        for t in terms:
            words.extend(re.findall(r"[A-Za-z\-]+", t.lower()))
    return Counter(words).most_common(topn)


def get_top_heads_from_list(reaction_list, topn=20):
    heads = []
    for reaction_text in reaction_list:
        if not isinstance(reaction_text, str):
            continue
        terms = [t.strip() for t in reaction_text.split(";") if t.strip()]
        for t in terms:
            tokens = re.findall(r"[A-Za-z\-]+", t.lower())
            if tokens:
                heads.append(tokens[-1])
    return Counter(heads).most_common(topn)


def get_top_phrases_from_list(reaction_list, topn=20):
    phrases = []
    for reaction_text in reaction_list:
        if not isinstance(reaction_text, str):
            continue
        terms = [t.strip().lower() for t in reaction_text.split(";") if t.strip()]
        phrases.extend(terms)
    return Counter(phrases).most_common(topn)


In [44]:
# EXPOSRES
kratom_only_df = df[df["All_Products_Clean"] == "Herbals\Mitragynine"]
seven_oh_df   = df[df["All_Products_Clean"] == "7-Hydroxymitragynine"]
pseudo_df     = df[df["All_Products_Clean"] == "Mitragynine Pseudoindoxyl"]

# Get top reactions per group using the same list-based functions
kratom_top_phrases = get_top_phrases_from_list(kratom_only_df["Reactions"].dropna(), topn=20)
seven_oh_top_phrases = get_top_phrases_from_list(seven_oh_df["Reactions"].dropna(), topn=20)
pseudo_top_phrases   = get_top_phrases_from_list(pseudo_df["Reactions"].dropna(), topn=20)


In [45]:

comparison_df = pd.DataFrame({
    "kratom": dict(kratom_top_phrases),
    "7oh": dict(seven_oh_top_phrases),
    "pseudo": dict(pseudo_top_phrases)
}).fillna(0)

print(comparison_df.head(10))


                            kratom   7oh  pseudo
drug dependence              116.0  11.0     0.0
death                         83.0   0.0     0.0
withdrawal syndrome           67.0   5.0     0.0
toxicity to various agents    52.0   0.0     0.0
vomiting                      49.0   0.0     0.0
seizure                       38.0   0.0     0.0
nausea                        34.0   2.0     0.0
substance use                 28.0   0.0     0.0
diarrhoea                     26.0   0.0     0.0
anxiety                       26.0   2.0     0.0


## Run again for outcomes

In [46]:
##OUTCOMES PATTERN

def get_top_words_from_list_o(reaction_list, topn=20):
    words = []
    for reaction_text in reaction_list:
        if not isinstance(reaction_text, str):
            continue
        terms = [t.strip() for t in reaction_text.split(";") if t.strip()]
        for t in terms:
            words.extend(re.findall(r"[^;]+", t.lower()))
    return Counter(words).most_common(topn)


def get_top_heads_from_list_o(reaction_list, topn=20):
    heads = []
    for reaction_text in reaction_list:
        if not isinstance(reaction_text, str):
            continue
        terms = [t.strip() for t in reaction_text.split(";") if t.strip()]
        for t in terms:
            tokens = re.findall(r"[^;]+", t.lower())
            if tokens:
                heads.append(tokens[-1])
    return Counter(heads).most_common(topn)


def get_top_phrases_from_list_o(reaction_list, topn=20):
    phrases = []
    for reaction_text in reaction_list:
        if not isinstance(reaction_text, str):
            continue
        terms = [t.strip().lower() for t in reaction_text.split(";") if t.strip()]
        phrases.extend(terms)
    return Counter(phrases).most_common(topn)


In [47]:
kratom_top_phrases_o = get_top_phrases_from_list_o(kratom_only_df["Outcomes"].dropna(), topn=20)

In [48]:

kratom_top_phrases_o = get_top_phrases_from_list_o(kratom_only_df["Outcomes"].dropna(), topn=20)
seven_oh_top_phrases_o = get_top_phrases_from_list_o(seven_oh_df["Outcomes"].dropna(), topn=20)
pseudo_top_phrases_o = get_top_phrases_from_list_o(pseudo_df["Outcomes"].dropna(), topn=20)

comparison_outcomes_df = pd.DataFrame({
    "kratom": dict(kratom_top_phrases_o),
    "7oh": dict(seven_oh_top_phrases_o),
    "pseudo": dict(pseudo_top_phrases_o)
}).fillna(0)

print(comparison_outcomes_df.head(10))

                       kratom  7oh  pseudo
died                      178    2     0.0
other outcomes            147   12     0.0
hospitalized              108   10     0.0
required intervention      76   10     0.0
life threatening           72    6     0.0
disabled                   67    3     0.0
non-serious                46    4     0.0


## MEGACODE: Use fuzzy matching to expand the standard reactions/reasons for use/ outcomes 

Normalize and return a more comphrehensive summary for each 




<span style="background-color: lightblue; padding: 5px;"><b>*I Used ChatGPT To help me use .update/.expand/.most_common below:*</b></span>

In [50]:
# the outputs
from rapidfuzz import process
SIMILARITY_THRESHOLD = 85  # fuzzy matching threshold

# Starting standard lists (expandable)
standard_reactions = ["Overdose", "Death", "Nausea", "Vomiting", "Seizure", "Drug Withdrawal Syndrome"]
standard_reasons   = ["Pain Relief", "Recreational", "Substance Abuse", "Anxiety"]
standard_outcomes  = ["Died", "Life Threatening", "Other Outcomes"]

# ------------------------------
# Functions
# ------------------------------
def expand_standard_lists(df, columns_dict):
    """Automatically expand standard lists from dataset."""
    expanded_lists = {}
    
    for col, standard_list in columns_dict.items():
        new_phrases = set()
        for text in df[col].dropna():
            phrases = [t.strip() for t in text.split(";") if t.strip()]
            new_phrases.update(phrases)
        
        # Lowercase for consistency
        new_phrases = {p.lower() for p in new_phrases}
        standard_list_lower = {p.lower() for p in standard_list}
        
        # Merge and convert back to list
        expanded_lists[col] = list(standard_list_lower.union(new_phrases))
    
    return expanded_lists

def extract_phrases(column):
    """Split semicolon-separated text, lowercase and strip."""
    all_phrases = []
    for text in column.dropna():
        phrases = [t.strip().lower() for t in text.split(";") if t.strip()]
        all_phrases.extend(phrases)
    return all_phrases

def fuzzy_standardize(phrases, standard_list):
    """Map phrases to closest match in standard_list using fuzzy matching."""
    standardized = []
    for p in phrases:
        match = process.extractOne(p, standard_list)
        if match and match[1] >= SIMILARITY_THRESHOLD:
            standardized.append(match[0])
        else:
            standardized.append(p)  # keep original if no good match
    return standardized

def get_top_counts(df, column, standard_list, topn=20):
    """Extract top normalized phrases for a column."""
    phrases = extract_phrases(df[column])
    phrases_std = fuzzy_standardize(phrases, standard_list)
    return Counter(phrases_std).most_common(topn)


columns_dict = {
    "Reactions": standard_reactions,
    "Reason for Use": standard_reasons,
    "Outcomes": standard_outcomes
}

expanded_lists = expand_standard_lists(df, columns_dict)

# Extract top counts

top_reactions = get_top_counts(df, "Reactions", expanded_lists["Reactions"])
top_reasons   = get_top_counts(df, "Reason for Use", expanded_lists["Reason for Use"])
top_outcomes  = get_top_counts(df, "Outcomes", expanded_lists["Outcomes"])

# Print summaries

print("Top Reactions:", top_reactions)
print("Top Reason for Use:", top_reasons)
print("Top Outcomes:", top_outcomes)


Top Reactions: [('toxicity to various agents', 320), ('drug abuse', 199), ('drug dependence', 190), ('drug interaction', 168), ('death', 143), ('withdrawal syndrome', 126), ('serotonin syndrome', 101), ('seizure', 88), ('vomiting', 85), ('overdose', 69), ('nausea', 60), ('drug withdrawal syndrome', 59), ('anxiety', 57), ('substance use', 57), ('unresponsive to stimuli', 54), ('completed suicide', 53), ('dependence', 52), ('cardiac arrest', 50), ('electrocardiogram qt prolonged', 45), ('accidental death', 44)]
Top Reason for Use: [('product used for unknown indication', 568), ('anxiety', 132), ('pain', 130), ('depression', 80), ('drug abuse', 44), ('drug dependence', 40), ('substance use', 31), ('back pain', 24), ('asthenia', 20), ('insomnia', 18), ('drug withdrawal syndrome', 15), ('menopausal symptoms', 15), ('covid-19', 15), ('phytotherapy', 14), ('hypertension', 12), ('drug use disorder', 12), ('stress', 11), ('major depression', 11), ('bipolar disorder', 11), ('analgesic therapy', 

## Create expanded list using custom function above

In [51]:
# Assuming top_reactions, top_reasons, top_outcomes are lists of tuples like [(phrase, count), ...]
reaction_list = [phrase for phrase, count in top_reactions]
reason_list   = [phrase for phrase, count in top_reasons]
outcome_list  = [phrase for phrase, count in top_outcomes]

print("Reactions List:", reaction_list)
print("Reasons List:", reason_list)
print("Outcomes List:", outcome_list)


Reactions List: ['toxicity to various agents', 'drug abuse', 'drug dependence', 'drug interaction', 'death', 'withdrawal syndrome', 'serotonin syndrome', 'seizure', 'vomiting', 'overdose', 'nausea', 'drug withdrawal syndrome', 'anxiety', 'substance use', 'unresponsive to stimuli', 'completed suicide', 'dependence', 'cardiac arrest', 'electrocardiogram qt prolonged', 'accidental death']
Reasons List: ['product used for unknown indication', 'anxiety', 'pain', 'depression', 'drug abuse', 'drug dependence', 'substance use', 'back pain', 'asthenia', 'insomnia', 'drug withdrawal syndrome', 'menopausal symptoms', 'covid-19', 'phytotherapy', 'hypertension', 'drug use disorder', 'stress', 'major depression', 'bipolar disorder', 'analgesic therapy']
Outcomes List: ['died', 'other outcomes', 'hospitalized', 'life threatening', 'disabled', 'required intervention', 'non-serious', 'congenital anomaly']


<span style="color: red; font-size: 24px;">TOP REACTIONS, REASONS FOR USE, and OUTCOMES using Text Analysis</span>

Top Reactions: ['toxicity to various agents', 'drug abuse', 'drug dependence', 'drug interaction', 'death', 'withdrawal syndrome', 'serotonin syndrome', 'seizure', 'vomiting', 'overdose', 'nausea', 'drug withdrawal syndrome', 'anxiety', 'substance use', 'unresponsive to stimuli', 'completed suicide', 'dependence', 'cardiac arrest', 'electrocardiogram qt prolonged', 'accidental death']

Top Reason for Use:['product used for unknown indication', 'anxiety', 'pain', 'depression', 'drug abuse', 'drug dependence', 'substance use', 'back pain', 'asthenia', 'insomnia', 'drug withdrawal syndrome', 'menopausal symptoms', 'covid-19', 'phytotherapy', 'hypertension', 'drug use disorder', 'stress', 'major depression', 'bipolar disorder', 'analgesic therapy']


Top Outcomes:['died', 'other outcomes', 'hospitalized', 'life threatening', 'disabled', 'required intervention', 'non-serious', 'congenital anomaly']

In [52]:
df.head(10)

,Case ID,Suspect Product Names,Suspect Product Active Ingredients,Reason for Use,Reactions,Serious,Outcomes,Sex,Event Date,Latest FDA Received Date,...,has_misc_drugs_abuse,has_sleep,All_Products_Clean_List,All_Products_Clean_Cleaned,Reason for Use_List,Reason for Use_Cleaned,Outcomes_List,Outcomes_Cleaned,Reactions_List,Reactions_Cleaned
0,6669537,<NA>,Herbals\Mitragynine;Modafinil,Drug Withdrawal Syndrome;Somnolence,Somnolence;Drug Withdrawal Syndrome;Miosis;Gen...,Serious,Hospitalized,Male,-,6/3/2008,...,False,False,[Herbals\Mitragynine;Modafinil],Herbals\Mitragynine;Modafinil,[Drug Withdrawal Syndrome;Somnolence],Drug Withdrawal Syndrome;Somnolence,[Hospitalized],Hospitalized,[Somnolence;Drug Withdrawal Syndrome;Miosis;Ge...,Somnolence;Drug Withdrawal Syndrome;Miosis;Gen...
1,6959961,<NA>,Herbals\Mitragynine;Hydromorphone Hydrochlorid...,Dependence;Drug Withdrawal Syndrome;Hypervigil...,Incorrect Route Of Product Administration;Drug...,Serious,Hospitalized;Other Outcomes,Male,-,4/3/2009,...,False,False,"[Buprenorphine\Naloxone, Herbals\Mitragynine;H...",Buprenorphine\Naloxone | Herbals\Mitragynine;H...,[Dependence;Drug Withdrawal Syndrome;Hypervigi...,Dependence;Drug Withdrawal Syndrome;Hypervigil...,[Hospitalized;Other Outcomes],Hospitalized;Other Outcomes,[Incorrect Route Of Product Administration;Dru...,Incorrect Route Of Product Administration;Drug...
2,7900650,Promethazine,Promethazine Hydrochloride;Propylhexedrine;Mor...,<NA>,Toxicity To Various Agents;Drug Screen Positive,Serious,Other Outcomes;Died,Male,-,4/15/2011,...,False,False,"[Promethazine, Promethazine Hydrochloride;Prop...",Promethazine | Promethazine Hydrochloride;Prop...,[],,[Other Outcomes;Died],Other Outcomes;Died,[Toxicity To Various Agents;Drug Screen Positive],Toxicity To Various Agents;Drug Screen Positive
3,8083892,Venlafaxine,Venlafaxine Hydrochloride;Buprenorphine;Trimep...,Product Used For Unknown Indication,Overdose;Accidental Death;Brain Oedema;Pulmona...,Serious,Died,Male,-,8/2/2011,...,False,False,"[Venlafaxine, Venlafaxine Hydrochloride;Bupren...",Venlafaxine | Venlafaxine Hydrochloride;Bupren...,[Product Used For Unknown Indication],Product Used For Unknown Indication,[Died],Died,[Overdose;Accidental Death;Brain Oedema;Pulmon...,Overdose;Accidental Death;Brain Oedema;Pulmona...
4,8121566,<NA>,Trimeprazine Tartrate;Herbals\Mitragynine;Venl...,Toxicity To Various Agents,Pulmonary Congestion;Pulmonary Oedema;Toxicity...,Serious,Other Outcomes;Died,Male,-,8/24/2011,...,False,False,[Trimeprazine Tartrate;Herbals\Mitragynine;Ven...,Trimeprazine Tartrate;Herbals\Mitragynine;Venl...,[Toxicity To Various Agents],Toxicity To Various Agents,[Other Outcomes;Died],Other Outcomes;Died,[Pulmonary Congestion;Pulmonary Oedema;Toxicit...,Pulmonary Congestion;Pulmonary Oedema;Toxicity...
5,8121559,<NA>,Tramadol;Zopiclone;Venlafaxine Hydrochloride;U...,Toxicity To Various Agents,Cardiac Arrest;Loss Of Consciousness;Respirato...,Serious,Life Threatening;Hospitalized;Died,Female,-,8/24/2011,...,False,False,[Tramadol;Zopiclone;Venlafaxine Hydrochloride;...,Tramadol;Zopiclone;Venlafaxine Hydrochloride;U...,[Toxicity To Various Agents],Toxicity To Various Agents,[Life Threatening;Hospitalized;Died],Life Threatening;Hospitalized;Died,[Cardiac Arrest;Loss Of Consciousness;Respirat...,Cardiac Arrest;Loss Of Consciousness;Respirato...
6,8121551,<NA>,Diazepam;Amphetamine;Olanzapine;Norfluoxetine;...,Toxicity To Various Agents,Cardiac Arrest;Toxicity To Various Agents;Pulm...,Serious,Died;Other Outcomes,Female,-,8/24/2011,...,False,False,[Diazepam;Amphetamine;Olanzapine;Norfluoxetine...,Diazepam;Amphetamine;Olanzapine;Norfluoxetine;...,[Toxicity To Various Agents],Toxicity To Various Agents,[Died;Other Outcomes],Died;Other Outcomes,[Cardiac Arrest;Toxicity To Various Agents;Pul...,Cardiac Arrest;Toxicity To Various Agents;Pulm...
7,8121536,Venlafaxine,Venlafaxine Hydrochloride;Buprenorphine;Herbal...,Toxicity To Various Agents,Cardiac Arrest;Loss Of Consciousness;Pulmon

## What substances are most likely to cause a serious event with kratom?

In [83]:
## convert my "Serious" column to a binary

df["serious_flag"] = df["Serious"].map({"Serious": 1, "Non-Serious": 0})

df.head(10)

,Case ID,Suspect Product Names,Suspect Product Active Ingredients,Reason for Use,Reactions,Serious,Outcomes,Sex,Event Date,Latest FDA Received Date,...,All_Products_Clean_Cleaned,Reason for Use_List,Reason for Use_Cleaned,Outcomes_List,Outcomes_Cleaned,Reactions_List,Reactions_Cleaned,Serious_binary,All_Products_List,serious_flag
0,6669537,<NA>,Herbals\Mitragynine;Modafinil,Drug Withdrawal Syndrome;Somnolence,Somnolence;Drug Withdrawal Syndrome;Miosis;Gen...,Serious,Hospitalized,Male,-,6/3/2008,...,Herbals\Mitragynine;Modafinil,[Drug Withdrawal Syndrome;Somnolence],Drug Withdrawal Syndrome;Somnolence,[Hospitalized],Hospitalized,[Somnolence;Drug Withdrawal Syndrome;Miosis;Ge...,Somnolence;Drug Withdrawal Syndrome;Miosis;Gen...,1,"[Herbals, Mitragynine, Modafinil]",1
1,6959961,<NA>,Herbals\Mitragynine;Hydromorphone Hydrochlorid...,Dependence;Drug Withdrawal Syndrome;Hypervigil...,Incorrect Route Of Product Administration;Drug...,Serious,Hospitalized;Other Outcomes,Male,-,4/3/2009,...,Buprenorphine\Naloxone | Herbals\Mitragynine;H...,[Dependence;Drug Withdrawal Syndrome;Hypervigi...,Dependence;Drug Withdrawal Syndrome;Hypervigil...,[Hospitalized;Other Outcomes],Hospitalized;Other Outcomes,[Incorrect Route Of Product Administration;Dru...,Incorrect Route Of Product Administration;Drug...,1,"[Buprenorphine, Naloxone, Herbals, Mitragynine...",1
2,7900650,Promethazine,Promethazine Hydrochloride;Propylhexedrine;Mor...,<NA>,Toxicity To Various Agents;Drug Screen Positive,Serious,Other Outcomes;Died,Male,-,4/15/2011,...,Promethazine | Promethazine Hydrochloride;Prop...,[],,[Other Outcomes;Died],Other Outcomes;Died,[Toxicity To Various Agents;Drug Screen Positive],Toxicity To Various Agents;Drug Screen Positive,1,"[Promethazine, Promethazine Hydrochloride, Pro...",1
3,8083892,Venlafaxine,Venlafaxine Hydrochloride;Buprenorphine;Trimep...,Product Used For Unknown Indication,Overdose;Accidental Death;Brain Oedema;Pulmona...,Serious,Died,Male,-,8/2/2011,...,Venlafaxine | Venlafaxine Hydrochloride;Bupren...,[Product Used For Unknown Indication],Product Used For Unknown Indication,[Died],Died,[Overdose;Accidental Death;Brain Oedema;Pulmon...,Overdose;Accidental Death;Brain Oedema;Pulmona...,1,"[Venlafaxine, Venlafaxine Hydrochloride, Bupre...",1
4,8121566,<NA>,Trimeprazine Tartrate;Herbals\Mitragynine;Venl...,Toxicity To Various Agents,Pulmonary Congestion;Pulmonary Oedema;Toxicity...,Serious,Other Outcomes;Died,Male,-,8/24/2011,...,Trimeprazine Tartrate;Herbals\Mitragynine;Venl...,[Toxicity To Various Agents],Toxicity To Various Agents,[Other Outcomes;Died],Other Outcomes;Died,[Pulmonary Congestion;Pulmonary Oedema;Toxicit...,Pulmonary Congestion;Pulmonary Oedema;Toxicity...,1,"[Trimeprazine Tartrate, Herbals, Mitragynine, ...",1
5,8121559,<NA>,Tramadol;Zopiclone;Venlafaxine Hydrochloride;U...,Toxicity To Various Agents,Cardiac Arrest;Loss Of Consciousness;Respirato...,Serious,Life Threatening;Hospitalized;Died,Female,-,8/24/2011,...,Tramadol;Zopiclone;Venlafaxine Hydrochloride;U...,[Toxicity To Various Agents],Toxicity To Various Agents,[Life Threatening;Hospitalized;Died],Life Threatening;Hospitalized;Died,[Cardiac Arrest;Loss Of Consciousness;Respirat...,Cardiac Arrest;Loss Of Consciousness;Respirato...,1,"[Tramadol, Zopiclone, Venlafaxine Hydrochlorid...",1
6,8121551,<NA>,Diazepam;Amphetamine;Olanzapine;Norfluoxetine;...,Toxicity To Various Agents,Cardiac Arrest;Toxicity To Various Agents;Pulm...,Serious,Died;Other Outcomes,Female,-,8/24/2011,...,Diazepam;Amphetamine;Olanzapine;Norfluoxetine;...,[Toxicity To Various Agents],Toxicity To Various Agents,[Died;Other Outcomes],Died;Other Outcomes,[Cardiac Arrest;Toxicity To Various Agents;Pul...,Cardiac Arrest;Toxicity To Various Agents;Pulm...,1,"[Diazepam, Amphetamine, Olanzapine, Norfluoxet...",1
7,8121536,Venlafaxine,Venlafaxine Hydrochloride;Buprenorphine;Herbal...,Toxicity To Various Agents,Cardiac Arrest;Loss Of Consciousness;Pulmonary...,Serious,Died;Other Outcomes,Male,-,8/24/2011,...

In [84]:
##split the string of All_Products_clean for each case into a list

def split_products(text):
    if pd.isna(text):
        return []
    # Split on | or ; or backslash
    items = re.split(r'[|;\\]+', text)
    return [i.strip() for i in items if i.strip()]

df["All_Products_List"] = df["All_Products_Clean"].apply(split_products)



<span style="background-color: lightblue; padding: 5px;"><b>*I Used ChatGPT To help with this, it suggested .explode:*</b></span>

In [85]:
## using the "explode" function in pandas, separate out each substance and corresponding case into its own row for analysis
df_long = df.explode("All_Products_List")
df_long = df_long.rename(columns={"All_Products_List": "Substance"})


In [86]:
## compute seriousness rate per individual substance
serious_rate = (
    df_long.assign(product_item=df_long["Substance"].apply(lambda x: x[0] if isinstance(x, list) else x))
    .groupby("product_item")["serious_flag"]
    .mean()
    .reset_index(name="serious_rate")
)

serious_rate.sort_values("serious_rate", ascending=False).head(20)

,product_item,serious_rate
0,.Alpha.-Hydroxyetizolam,1.0
351,Oxycodone,1.0
367,Pepcid,1.0
364,Paroxetine Hydrochloride,1.0
363,Paroxetine,1.0
362,Parnate,1.0
361,Parafluorofentanyl,1.0
360,Pantoprazole,1.0
359,Pamelor,1.0
358,Paliperidone Palmitate,1.0


In [78]:
## ok, because 93% of cases are serious, this is not the right measure for this.

In [88]:
## rank substances by % of serious cases
substance_severity = (
    df_long.groupby("Substance")["serious_flag"]
    .agg(['mean', 'count'])
    .rename(columns={'mean': 'serious_rate', 'count': 'case_count'})
    .sort_values("case_count", ascending=False)
)


substance_severity.head(20)

##NOTE: im not filtering out small sample sizes because this dataset is not that large

,serious_rate,case_count
Substance,,
Herbals,0.929860,1497
Mitragynine,0.931432,1473
Fentanyl,0.986014,143
Gabapentin,0.946903,113
Bupropion,0.986486,74
Sertraline Hydrochloride,1.000000,69
Alcohol,0.985294,68
Acetaminophen,1.000000,67
Sertraline,1.000000,62


In [91]:
substance_severity['Pct_cases'] =substance_severity['case_count']/1503
substance_severity.head(10)
substance_severity.to_csv("Kratom_co-usedrugs_seriousrate.csv")


<span style="color: red; font-size: 24px;">By count, Fentanyl, Gabapentin, Bupropion, Sertraline Hydrochloride, and Alcohol were the top five substances most likely to produce a serious case when used with kratom</span> 
- See substance_severity for full list 

In [93]:
df_refs = pd.read_csv("Cleaned_NewCols_FAERS.csv")
df_refs['Literature Reference'].unique()

array(['-',
       'Neerman, M.F., Frost, R.E., Deking, J.. A Drug Fatality Involving Kratom. Journal of forensic sciences. 2013;58 (S1):S278-S279',
       'Mowry JB; Spyker DA, Cantilena Jr. LR, McMillan N, Ford M. 2013 Annual Report of the American Association of Poison Control Centers \x92 National Poison Data System (NPDS):31st Annual Report. CLINICAL TOXICOLOGY. 2014;52:1032-1283',
       'Mowry J.B., Spyker D.A., Cantilena L.R., Mcmillan N, Ford M. 2013Annual Report of the American Association of Poison Control Centers National Poison Data System (NPDS): 31st Annual Report. Clinical Toxicology. 2014;52:1032-1283',
       "Mowry J.B., Spyker D.A., Cantilena JR L.R., Mcmillan N., Ford M.. 2013 Annual Report of the American Association of Poison Control Centers' National Poison Data System (NPDS): 31st Annual Report. Clinical Toxicology. 2014;52 (10):1032-1283",
       'McIntyre IM, Trochta A, Stolberg S, Campman SC. Mitragynine kratom related fatality: a case report with postmortem

In [97]:
# Install the necessary library if you haven't already
# !pip install fuzzywuzzy python-Levenshtein  # or !pip install rapidfuzz

from rapidfuzz import fuzz, process
import pandas as pd
import numpy as np

references = df_refs['Literature Reference'].dropna().tolist()
unique_refs = []
duplicate_indices = []

# Set a similarity threshold (adjust as needed)
threshold = 90

for i, ref1 in enumerate(references):
    if i in duplicate_indices:
        continue
        
    unique_refs.append(ref1)
    
    # Compare with remaining references
    for j in range(i+1, len(references)):
        ref2 = references[j]
        # Calculate similarity ratio
        similarity = fuzz.ratio(ref1.lower(), ref2.lower())
        
        if similarity >= threshold:
            duplicate_indices.append(j)

# Create a new DataFrame with only unique references
unique_references = pd.DataFrame({'Literature Reference': unique_refs})
unique_references

,Literature Reference
0,-
1,"Neerman, M.F., Frost, R.E., Deking, J.. A Drug..."
2,"Mowry JB; Spyker DA, Cantilena Jr. LR, McMilla..."
3,"McIntyre IM, Trochta A, Stolberg S, Campman SC..."
4,"McIntyre I M.. Mitragynine ""Kratom' Related Fa..."
...,...
151,"Bursová M, Černá I, Horák V, Šňupárek Z. ÚMRTÍ..."
152,"Brogdon H D, McPhee M M, Paine M F, Cox E J an..."
153,"Breugelmans L, Boland A, Fernandez MDMR, Di Fa..."
154,"Breugelmans L, Boland A, Fernandez MD, Di Fazi..."


### NOTE FOR SANDEEP: I'm going to keep going with this analysis but I am going to stop for now! More to do on the references/correlations/demographics that could be useful
---